# **Source**:

https://www.kaggle.com/datasets/manjitbaishya001/airbnb-new-york-jan-2024?select=detailed_reviews.csv, sourced from Airbnb Description: Airbnb
data from New York focussing on listings, locations, and user reviews of
locations

# **What is the Dataset about?**:

This dataset contains listings of Airbnbs in New York, listing features such as what type of housing it is (Rental Unit, Loft, etc.) and the price of the Airbnb. Note that there are several rows with missing values in the price column, which are dealt with during Data Preprocessing. Other features include the name of the host, neighbourhood group, neighbourhood, latitude, longitude, minimum nights to stay, number of reviews about the establishment, the date of the last review, and reviews per month. These features can be used to estimate what an accurate valuation of the Airbnb might be.

There is also a dataset with detailed reviews for each Airbnb, which can be used to evaluate the quality of the listing based on user experiences.


# **Data Preprocessing**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

# importing dataset and organizing
listings_df = pd.read_csv("../dataset/listings.csv")

# dropping all listings with a NA price as the percentage of NAs is more than 5%
# and imputation on a feature as complex as a listing price seems unnecessary
# given the quanitity of listings that do have an associated price
listings_df = listings_df.dropna(subset=['price'])

# all values are in the correct format
print("Data Types: ", list(listings_df.dtypes))


print(3*"\n", "Dataframe:")

listings_df.head()

# **Data exploration and summary statistics**

## **Statistical Method #1**

**Null Hypothesis ($H_{0}$) :** The neighborhood group of the Airbnb does not have a statistically significant impact on the availability for the Airbnb.

**Alternative Hypothesis ($H_{a}$) :** The neighborhood group of the Airbnb has a statistically significant impact on the availability for the Airbnb.

**Alpha-Value ($a$) :** 0.05

**Confidence level:** 95%

In [ ]:
import numpy as np
from scipy.stats import kruskal

# Group by neighborhood group and calculate mean availability
df_groups = listings_df.groupby('neighbourhood_group')['availability_365'].mean().sort_values(ascending=False)

# Convert mean availability to a list
mean_availability = df_groups.tolist()

# Perform Kruskal-Wallis test to compare mean availability across neighborhood groups
statistic, p = kruskal(*[listings_df[listings_df['neighbourhood_group'] == group]['availability_365'] for group in df_groups.index])

print("Kruskal-Wallis Test Statistic:", statistic)
print("P-Value:", p)
print(df_groups)

# Plotting the graph
df_groups.plot.bar()
plt.ylabel('Average Availability (days)')
plt.xlabel('Neighbourhood Groups')
plt.title('Average Availabilities Across Neighbourhood Groups')
plt.show()

Since the p-value is lower than the confidence interval, we can reject the null hypothesis. This means that the location has a statistically significant impact on the availability for the Airbnb. The most available Airbnbs are located in Staten Island, and the least available Airbnbs are in Brooklyn.

This is important for the machine learning model because the model should take into account that there are disproportionate availabilities based on the location of the Airbnb. The pricing model will weight this as well.

## **Statistical Method #2**

**Null Hypothesis ($H_{0}$) :** The neighbourhoods in Manhattan do not have a statistically significant impact on the mean prices for Entire home/apt units.

**Alternative Hypothesis ($H_{a}$) :** The neighbourhoods in Manhattan have a statistically significant impact on the mean prices for Entire home/apt units.

**Alpha-Value ($a$) :** 0.05

**Confidence level:** 95%

In [ ]:
from scipy.stats import f_oneway

# Filter the dataset to include only listings in Manhattan with 'Entire home/apt' room type
df_manhattan = listings_df[(listings_df['neighbourhood_group'] == 'Manhattan') & (listings_df['room_type'] == 'Entire home/apt')]

# Group by neighborhood and calculate mean prices for 'Entire home/apt' units
df_groups = df_manhattan.groupby('neighbourhood')['price'].mean().sort_values(ascending=False)
mean_prices = list(df_groups)

# Perform one-way ANOVA test to test for statistically significant differences between the mean prices of neighborhoods
statistic, p = f_oneway(*[df_manhattan[df_manhattan['neighbourhood'] == neighborhood]['price'] for neighborhood in df_groups.index])

print("P-Value:", p)

# Plot the scatter plot
plt.scatter(df_groups.index, mean_prices)
plt.xticks(rotation=90)
plt.ylabel('Average Price ($)')
plt.xlabel('Manhattan Neighbourhoods')
plt.title('Average Prices of Entire home/apt units in Manhattan Neighbourhoods')
plt.show()

As the p-value (7.9997e-22) is less than the alpha value of 0.05, we can reject the null hypothesis and conclude that there is a significant difference in the prices of Entire home/apt units between Manhattan neighbourhoods. Tribeca is the most expensive location in Manhattan based on the data provided/present for entire home/apts, with an average price of approximately $595. These findings allude to the fact that New York's neighbourhood groups have a varying price range of entire home/apt units throught their neighbourhoods.

## **Statistical Method #3**

**Null Hypothesis ($H_{0}$) :** The number of reviews of an Airbnb does not have a statistically significant impact on the price of the Airbnb.

**Alternative Hypothesis ($H_{a}$) :** The number of reviews of an Airbnb has a statistically significant impact on the price of the Airbnb.

**Alpha-Value ($a$) :** 0.05

**Confidence level:** 95%

In [ ]:
from scipy import stats

# using a threshold that listings with a number of reviews of lesser than 50 is
# regarded as 'Low' number of reviews and those with higher than 50 is regarded
# as 'High' number of reviews.

# taking out outliers (only 7 listings have prices higher than 19000 and may
# ruin integrity of our dataset)
new_df = listings_df[listings_df["price"] < 19000.0]

low_df = new_df[new_df['number_of_reviews'] < 50]
high_df = new_df[new_df['number_of_reviews'] >= 50]

low_prices = low_df['price']
high_prices = high_df['price']

t_statistic, p_value = stats.ttest_ind(low_prices, high_prices)

print("T-statistic:", t_statistic)
print("P-value:", p_value)

# plotting the graph
sns.scatterplot(data=new_df, x="price", y="number_of_reviews", linewidth = 0.5, palette="mako_r", alpha=1)
plt.xscale('log')


The p-value of 1.444e-17 is extremely small, indicating strong evidence against the null hypothesis.

Given the very small p-value, much smaller than the typical significance level of 0.05, we reject the null hypothesis. Therefore, we conclude that there is a statistically significant difference in mean prices between the low and high review groups.

If there's a significant difference in mean prices between listings with low and high review counts, this finding could have implications for pricing strategy. For example, it might suggest that listings with higher review counts can command higher prices, potentially reflecting greater perceived value among customers.

# **Initial Conclusions through Exploratory Analysis**

Through our exploratory data analysis and basic data cleaning (the data was pretty clean to begin with) we hope to build a machine learning model to predict the price of Airbnb listings based on features such as location, property type, number of bedrooms, amenities, and historical booking data. This could help hosts optimize their pricing strategy and maximize their revenue.

Through our aforementioned Hypothesis Testing, we have found correlatory evidence in features provided in the dataset and being able to predict the value of a listing. This provides a positive outlook onto being able to create a Machine Learning pricing model in the future.

### imports


In [ ]:
import os
import tarfile
import requests
import torch
from torch.utils.data import Dataset
import pandas as pd
import cv2
import os
import torch.nn as nn
import torch.nn.functional as F
import math
import tqdm
from sklearn.model_selection import train_test_split
import copy
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.optim as optim

### model definition


In [ ]:
#

class Net(nn.Module):
  def __init__(self, n_classes):
    super().__init__()

    self.conv = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=4, stride=1, padding=1)

    self.fc1 = nn.Linear(8 * 6 * 7, n_classes)
    self.fc2 = nn.Linear(n_classes, 1)

  def forward(self, x):
    x = x.unsqueeze(1)
    x = self.conv(x)
    x = F.relu(x)
    x = x.view(x.size(0), -1)

    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    return x


model = nn.Sequential(
    nn.Linear(8, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

In [ ]:
n_epochs = 100
batch_size = 10

### train model using train and validation datasets

In [ ]:
# loss function and optimizer
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# train-test split of the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, shuffle=True)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

train_size = int(0.8 * len(dataset))
test_size = int(0.1 * len(dataset))
val_size = int(0.1 * len(dataset))
train_set, test_set, val_set = torch.utils.data.random_split(dataset, [train_size, test_size, val_size])

y_train = train_set.pop('price')
y_test = test_set.pop('price')
y_val = val_set.pop('price')

x_train = torch.tensor(train_set, dtype=torch.float32)
x_test = torch.tensor(test_set, dtype=torch.float32)
x_val = torch.tensor(val_set, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
y_val = torch.tensor(y_val, dtype=torch.float32).reshape(-1, 1)

# Hold the best model
best_mse = np.inf   # init to infinity
best_model = None
mse_store = []
loss_store = []
eval_epochs = 0
batch_start = torch.arange(0, len(X_train), batch_size)

# training loop
for epoch in range(n_epochs):

    model.train()

    with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
        bar.set_description(f"Epoch {epoch}")

        for start in bar:

            # take a batch
            X_batch = X_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]

            # forward pass
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)

            # backward pass
            optimizer.zero_grad()
            loss.backward()

            # update weights
            optimizer.step()

            # print progress
            bar.set_postfix(mse=float(loss))

    # evaluate accuracy at end of each epoch
    model.eval()
    y_pred = model(x_val)

    eval_epochs += 1

    mse = loss_fn(y_pred, y_val)

    mse = float(mse)
    mse_store.append(mse)
    loss_store.append(loss.item())

    print(f"Epoch: {epoch}, Train Loss: {loss.item()} Eval Loss: {mse}")

    if mse < best_mse:
        best_mse = mse
        best_model = copy.deepcopy(model.state_dict())

# Save your model's weights
model.load_state_dict(best_model)
torch.save(model.state_dict(), "model.pth")

### plot training and eval loss values




In [ ]:
# restore model and return best accuracy
model.load_state_dict(best_model)

In [ ]:
# Plot the training and validation loss
plt.plot(eval_epochs, loss_store, label='Training loss')
plt.plot(eval_epochs, mse_store, label='Validation loss')
plt.legend()
plt.show()

# testing model on test set


In [ ]:
# Load from your saved model using torch.load
model_state_dict = torch.load("./model.pth")

model = nn.Sequential(
    nn.Linear(8, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

model.load_state_dict(model_state_dict)

# set model to inference mode
model.eval()
batch_size = 1
preds = []

test_loader = DataLoader(x_test, batch_size=batch_size, shuffle=False)


with torch.no_grad():

  for x in tqdm(test_loader):

    y_pred = model(x)

    preds.append(y_pred)


# Get the true labels for the validation dataset
true_y = torch.tensor(y_test)
preds = torch.tensor(preds)

accuracy = (true_y == preds).float().mean().item()
print(f"Accuracy: {accuracy}")